## Using Feature Flags to Test Faster Data Processing Techniques

I have two functions which search for the longest strings in a large datasets.
I will use feature flags to switch between them, testing which is more efficient. 

Function 1, Simple:
1. Measures length of each string
2. Returns string with longest length

Function 2, MapReduce:
1. You split your dataset up into small chunks
2. Find longest string in each chunk separately
3. Group chunks back together and process the smaller and simpler reduced output 

### Imports

In [7]:
import time
from functools import reduce

### Feature Flag Library Import

In [8]:
#feature flag client
import ldclient
from ldclient.config import Config

#test project SDK key
ldclient.set_config(Config("sdk-a3b73679-1452-4129-9909-ec5759094fa2"))
ld_client = ldclient.get()

### Functions

In [9]:
#### Simple Function
def longest_strings(string):
    longest_string = None
    Longest_length = 0
    for s in string:
        if len(s) > Longest_length:
            longest_string = s
            Longest_length = len(s)
    return longest_string

##### map reduce functions

def reducer(p, c):
    if p[1] > c[1]:
        return p
    return c

def mapper(chunk):
    list_of_lengths = map(len, chunk)
    mapped = zip(chunk, list_of_lengths)
    return reduce(reducer, mapped)

#split list into N number of other lists
def chunker_list(seq, size):
    return (seq[i::size] for i in range(size))

### Variables

In [13]:
string_example = ["Python", "apple", "Alex long"]*1000000

## Feature Flag OFF

In [15]:
show_feature = ld_client.variation("MapReduce", {"key": "chris@test.com"}, False)

if show_feature:
    t2 = time.process_time()
    data_chunks = chunker_list(string_example, 2)

    #step 1
    from Mapper import mapper
    mapped = map(mapper, data_chunks)

    #step 2
    reduced = reduce(reducer, mapped)
    print("MapReduce Feature On:   "+str(reduced))
    t3 = time.process_time()
    print(t3-t2)
    
    
else:
    t0 = time.process_time()
    print("MapReduce Feature Off:  "+longest_strings(string_example))
    t1 = time.process_time()
    print(t1-t0)
    

MapReduce Feature Off:  Alex long
0.28125


## Feature Flag ON

In [20]:
show_feature = ld_client.variation("MapReduce", {"key": "chris@test.com"}, False)

if show_feature:
    t2 = time.process_time()
    data_chunks = chunker_list(string_example, 2)

    #step 1
    from Mapper import mapper
    mapped = map(mapper, data_chunks)

    #step 2
    reduced = reduce(reducer, mapped)
    print("MapReduce Feature On:   "+str(reduced))
    t3 = time.process_time()
    print(t3-t2)
    
    
else:
    t0 = time.process_time()
    print("MapReduce Feature Off:  "+longest_strings(string_example))
    t1 = time.process_time()
    print(t1-t0)
    

MapReduce Feature On:   ('Alex long', 9)
0.640625


## Conclusion

1. Our Feature Flag works
2. MapReduce is slower, this is because usually we process the chunks in parallel with each logical CPU - for simplicity i did not do this for the demonstration